#통합 책 추천 시스템

- 고정 userid = 88705 / 264321 / 182459 / 161936 / 226482 / random 1 : 121184/ random 2 -> 204167
/ 제일 부정적인 사용자
- 10가지의 방식을 활용하여 user에게 추천한다.

1. 베스트셀러/작가 - 허재훈
1. 베스트셀러/지역 - 백경관
1. 콘텐츠베이스 - 책 제목 - 송도훈
1. 협력필터링 / KNN-cosine 3가지 방식 - 허재훈
1. 행렬분해 / SVD latent 150 - 허재훈
1. 행렬분해 / NMF - 송도훈
1. 행렬분해 / ALS - 허재훈

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
book_rating = pd.read_csv('/content/drive/MyDrive/bx/BX-Book-Ratings.csv',
                          sep = ';', encoding = 'latin_1', error_bad_lines = False)
book_rating.drop(index = book_rating[book_rating['Book-Rating'] == 0].index, inplace = True) #평점 0점은 결측치 처리
book_rating

<ipython-input-3-e9cc15fc3536>:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  book_rating = pd.read_csv('/content/drive/MyDrive/bx/BX-Book-Ratings.csv',


,User-ID,ISBN,Book-Rating
1,276726,0155061224,5
3,276729,052165615X,3
4,276729,0521795028,6
6,276736,3257224281,8
7,276737,0600570967,6
...,...,...,...
1149773,276704,0806917695,5
1149775,276704,1563526298,9
1149777,276709,0515107662,10
1149778,276721,0590442449,10


In [ ]:
book_info = pd.read_csv('/content/drive/MyDrive/bx/BX-Books.csv',
                        sep = ';', encoding = 'latin_1', error_bad_lines = False)
# 불필요한 URL, 출판사 값 버리기
book_info.drop(columns = ['Image-URL-S','Image-URL-M', 'Image-URL-L', 'Publisher'], inplace = True)
book_info

<ipython-input-4-9cc67933f6d9>:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  book_info = pd.read_csv('/content/drive/MyDrive/bx/BX-Books.csv',
b'Skipping line 6452: expected 8 fields, saw 9\nSkipping line 43667: expected 8 fields, saw 10\nSkipping line 51751: expected 8 fields, saw 9\n'
b'Skipping line 92038: expected 8 fields, saw 9\nSkipping line 104319: expected 8 fields, saw 9\nSkipping line 121768: expected 8 fields, saw 9\n'
b'Skipping line 144058: expected 8 fields, saw 9\nSkipping line 150789: expected 8 fields, saw 9\nSkipping line 157128: expected 8 fields, saw 9\nSkipping line 180189: expected 8 fields, saw 9\nSkipping line 185738: expected 8 fields, saw 9\n'
b'Skipping line 209388: expected 8 fields, saw 9\nSkipping line 220626: expected 8 fields, saw 9\nSkipping line 227933: expected 8 fields, saw 11\nSkipping line 228957: expected 8 fields, saw 10\nSkipping line 245933: expec

,ISBN,Book-Title,Book-Author,Year-Of-Publication
0,0195153448,Classical Mythology,Mark P. O. Morford,2002
1,0002005018,Clara Callan,Richard Bruce Wright,2001
2,0060973129,Decision in Normandy,Carlo D'Este,1991
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999
...,...,...,...,...
271355,0440400988,There's a Bat in Bunk Five,Paula Danziger,1988
271356,0525447644,From One to One Hundred,Teri Sloat,1991
271357,006008667X,Lily Dale : The True Story of the Town that Ta...,Christine Wicker,2004
271358,0192126040,Republic (World's Classics),Plato,1996


In [ ]:
# 결측치가 매우 소수의 행에 존재하므로 결측치가 있는 행만 제거 - 도훈님 코드
book_info.dropna(inplace=True)
book_info.reset_index(drop=True, inplace=True)
book_info

,ISBN,Book-Title,Book-Author,Year-Of-Publication
0,0195153448,Classical Mythology,Mark P. O. Morford,2002
1,0002005018,Clara Callan,Richard Bruce Wright,2001
2,0060973129,Decision in Normandy,Carlo D'Este,1991
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999
...,...,...,...,...
271354,0440400988,There's a Bat in Bunk Five,Paula Danziger,1988
271355,0525447644,From One to One Hundred,Teri Sloat,1991
271356,006008667X,Lily Dale : The True Story of the Town that Ta...,Christine Wicker,2004
271357,0192126040,Republic (World's Classics),Plato,1996


In [ ]:
book_user = pd.read_csv('/content/drive/MyDrive/bx/BX-Users.csv', sep = ';', encoding = 'latin_1', error_bad_lines = False)
book_user

<ipython-input-6-e195780cffae>:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  book_user = pd.read_csv('/content/drive/MyDrive/bx/BX-Users.csv', sep = ';', encoding = 'latin_1', error_bad_lines = False)


,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN
...,...,...,...
278853,278854,"portland, oregon, usa",NaN
278854,278855,"tacoma, washington, united kingdom",50.0
278855,278856,"brampton, ontario, canada",NaN
278856,278857,"knoxville, tennessee, usa",NaN


In [ ]:
# 88705 / 254321 / 182459 / 161936 / 226482

book_user[book_user['User-ID']==226482]

,User-ID,Location,Age
226481,226482,"singapore, n/a,",21.0


##유저 분석

###가장 나쁜 평점을 준 랜덤유저1(총 평점갯수 대비)

In [ ]:
bad_rate_user = book_rating[book_rating['Book-Rating'] < 4] #평점 4점 이하인 유저
bad_rate_arr = bad_rate_user['User-ID'].value_counts().loc[lambda x : x >=20] #평점 갯수가 20개 이상인 유저

a = list(bad_rate_arr.index)
b = list(bad_rate_arr)
bad_rate_dict = {}

for id, count in zip(a,b): #딕셔너리 형태로 분류
    bad_rate_dict[id] = count
bad_rate_dict

{11676: 462,
 264321: 54,
 76499: 38,
 35836: 33,
 23872: 29,
 35859: 28,
 229551: 27,
 248718: 26,
 218552: 26,
 204167: 25,
 157247: 25,
 236757: 24,
 38273: 23,
 89891: 20,
 233911: 20}

In [ ]:
bad_ratio = []

for x in bad_rate_arr.index: # 위 딕셔너리의 키값과 동일
    num = book_rating[book_rating['User-ID']==x]['User-ID'].count() #해당 유저의 총 평점 갯수
    bad_ratio.append(round(bad_rate_dict[x]/num, 5)) #나쁜 평점 수 / 총 평점 수
bad_ratio # 해당 사용자의 평점 중, bad 평점의 비율

[0.0542,
 0.18947,
 0.03668,
 0.30556,
 0.05709,
 0.03378,
 0.18367,
 0.02743,
 0.1844,
 0.52083,
 0.10163,
 0.12698,
 0.07372,
 0.1,
 0.15748]

In [ ]:
bad_ratio_df = pd.DataFrame(bad_ratio, index = bad_rate_arr.index)
bad_ratio_df
# 204167 유저가 남긴 평점 대비 가장 부정적인 평점을 남긴 사람이다.

,0
11676,0.05420
264321,0.18947
76499,0.03668
35836,0.30556
23872,0.05709
35859,0.03378
229551,0.18367
248718,0.02743
218552,0.18440
204167,0.52083


In [ ]:
#204167 유저의 평점 확인
user_204167_rate = book_rating[book_rating['User-ID']==204167]
user_204167_rate.sort_values('Book-Rating',ascending = False)

,User-ID,ISBN,Book-Rating
843204,204167,067173685X,8
843203,204167,0671736809,8
843182,204167,067102020X,7
843044,204167,0373709277,7
843115,204167,0441003389,6
843234,204167,1551660717,5
843017,204167,0373034091,5
843112,204167,0440241057,5
843102,204167,0425158624,5
843146,204167,0451400933,5


### 평균 이하의 평점 갯수를 준 랜덤 유저2(사실은 한국인 중 가장 많은 평점을 준사람)

In [ ]:
korea_user_id = book_user[book_user['Location'].str.contains('korea')]['User-ID']
korea_user_id

5594        5595
6353        6354
6658        6659
7750        7751
10335      10336
           ...  
273775    273776
273790    273791
273798    273799
275278    275279
275845    275846
Name: User-ID, Length: 236, dtype: int64

In [ ]:
book_rating[book_rating['User-ID'].isin(korea_user_id)].value_counts('User-ID')[:5]

User-ID
121184    7
47465     6
156264    4
133052    4
129655    3
dtype: int64

In [ ]:
user_121184_rate = book_rating[book_rating['User-ID']==121184] #한국에서 가장 많이 평점을 준
user_121184_rate

,User-ID,ISBN,Book-Rating
501233,121184,0312995423,10
501234,121184,0399138250,9
501235,121184,0399147888,10
501236,121184,0425133516,9
501242,121184,05151324809,10
501243,121184,0515136565,8
501246,121184,0895261480,10


##KNN cosine 방식

In [ ]:
book_rating_info = pd.merge(book_rating, book_info[['ISBN', 'Book-Title']], on = 'ISBN')
book_rating_info #book_rating에 title 추가하기

,User-ID,ISBN,Book-Rating,Book-Title
0,276726,0155061224,5,Rites of Passage
1,276729,052165615X,3,Help!: Level 1
2,276729,0521795028,6,The Amsterdam Connection : Level 4 (Cambridge ...
3,276744,038550120X,7,A Painted House
4,11676,038550120X,10,A Painted House
...,...,...,...,...
383836,276688,0425156737,2,Graven Images
383837,276688,0671563149,6,MUDDY WATER (Peter Bartholomew Mysteries)
383838,276688,1575660792,7,Gray Matter
383839,276704,0380796155,5,White Abacus


In [ ]:
# 행렬 대입전 전처리 과정

user_ids = book_rating_info["User-ID"].unique().tolist()

user2idx_dict = {x: i for i, x in enumerate(user_ids)}
idx2user_dict = {i: x for i, x in enumerate(user_ids)}

book_ids = book_rating_info["ISBN"].unique().tolist()

book2idx_dict = {x: i for i, x in enumerate(book_ids)}
idx2book_dict = {i: x for i, x in enumerate(book_ids)}

book_rating_info["user_idx"] = book_rating_info["User-ID"].map(user2idx_dict)
book_rating_info["book_idx"] = book_rating_info["ISBN"].map(book2idx_dict)

num_users = book_rating_info["User-ID"].nunique()
num_books = book_rating_info["ISBN"].nunique()
print(num_users, num_books)

book_rating_info = book_rating_info.sort_values(['user_idx', 'book_idx'])
book_rating_info = book_rating_info.reset_index(drop=True)
book_rating_info

68091 149835


,User-ID,ISBN,Book-Rating,Book-Title,user_idx,book_idx
0,276726,0155061224,5,Rites of Passage,0,0
1,276729,052165615X,3,Help!: Level 1,1,1
2,276729,0521795028,6,The Amsterdam Connection : Level 4 (Cambridge ...,1,2
3,276744,038550120X,7,A Painted House,2,3
4,11676,038550120X,10,A Painted House,3,3
...,...,...,...,...,...,...
383836,276660,0583307841,8,ROBOT RACE (MICRO ADV 6),68089,149803
383837,276664,0004703723,9,Dictionary Of Economics-2Nd Ed,68090,149804
383838,276664,0140136908,7,History of Economic Thought (Penguin Economics),68090,149805
383839,276664,0631189629,9,British Social Policy Since 1945 (Making Conte...,68090,149806


In [ ]:
# 희소행렬
from scipy.sparse import coo_matrix, linalg

rating_sparse_arr = coo_matrix((book_rating_info['Book-Rating'],
                                (book_rating_info['user_idx'],
                                 book_rating_info['book_idx'])),
                               shape = (num_users, num_books))
rating_sparse_arr

<68091x149835 sparse matrix of type '<class 'numpy.int64'>'
	with 383841 stored elements in COOrdinate format>

In [ ]:
rating_sparse_arr = rating_sparse_arr.astype(np.float32)

In [ ]:
#KNN 방식의 cosine 유사도 사용
from sklearn.neighbors import NearestNeighbors
knn_model = NearestNeighbors(n_neighbors = 30, metric = 'cosine')
knn_model.fit(rating_sparse_arr)

NearestNeighbors(metric='cosine', n_neighbors=30)

###타겟 유저

In [ ]:
# 타겟 유저 설정
target_user_id = 88705

In [ ]:
target_user_df = book_rating_info[book_rating_info['User-ID'] == target_user_id] #타겟 유저의 데이터프레임 생성
#희소행렬 만들기
target_user_sp = coo_matrix((target_user_df['Book-Rating'].to_list(),
                             ([0]* len(target_user_df),
                              target_user_df['book_idx'].to_list())),
                            shape = (1, num_books))
target_user_sp

<1x149835 sparse matrix of type '<class 'numpy.int64'>'
	with 1 stored elements in COOrdinate format>

In [ ]:
# distance와 index로 분리 neighbor값은 30(가까운 유저 30명)
dist, idx = knn_model.kneighbors(target_user_sp, n_neighbors = 30)

In [ ]:
# 가까운 값 30개 및 10643(타겟 index)와의 거리 확인
for d, i in zip(dist[0], idx[0]):
    print(i, round(d,4)) #거리가 0은 본인 인덱스, 거리가 가까울 수록 가까운 친구들

57927 0.0
45399 1.0
45402 1.0
45404 1.0
45403 1.0
45400 1.0
45409 1.0
45410 1.0
45411 1.0
45412 1.0
45413 1.0
45407 1.0
45408 1.0
45384 1.0
45398 1.0
45397 1.0
45396 1.0
45395 1.0
45394 1.0
45393 1.0
45392 1.0
45391 1.0
45390 1.0
45389 1.0
45388 1.0
45387 1.0
45386 1.0
45401 1.0
45382 1.0
45405 1.0


In [ ]:
# 타겟 자신을 제외한 유사 유저의 인덱스 받아오기
friends_idx = idx[0][1:]
friends_idx

array([45399, 45402, 45404, 45403, 45400, 45409, 45410, 45411, 45412,
       45413, 45407, 45408, 45384, 45398, 45397, 45396, 45395, 45394,
       45393, 45392, 45391, 45390, 45389, 45388, 45387, 45386, 45401,
       45382, 45405])

In [ ]:
#영화 추천
friend_df = book_rating_info[book_rating_info['user_idx'].isin(friends_idx)] #유사 유저들의 df
friend_df = friend_df[friend_df['Book-Rating'] > 3] #추천 하는 목적이므로 평점 3점 초과값만 받기
friend_df

,User-ID,ISBN,Book-Rating,Book-Title,user_idx,book_idx
352208,224934,0060927569,9,High Tide in Tucson : Essays from Now or Never,45382,24073
352209,224934,0671793888,9,Cowboys Are My Weakness,45382,31929
352211,191346,0345323750,6,The Life and Loves of a She-Devil,45384,24083
352213,249738,0670858692,7,Rose Madder,45386,24084
352214,117796,0890438439,5,How to Clean Practically Anything,45387,24086
...,...,...,...,...,...,...
352274,209257,0399150412,6,White Death: A Novel from the Numa Files,45411,66056
352275,209257,0812542967,6,Lodestar (Firestar Saga (Paperback)),45411,126668
352276,252799,1861976127,8,"Eats, Shoots and Leaves: The Zero Tolerance Ap...",45412,24140
352277,252799,0099284677,9,Stoned,45412,142691


In [ ]:
#빈도 기반 추천 - 나와 유사한 사람들이 가장 많이 본 책
friend_count_df = friend_df.groupby('ISBN').agg('count').sort_values(
    'Book-Rating', ascending = False)['Book-Rating'] #깔끔하게 df로 표현하기 위해 Book-Rating만 가져오기
friend_count_df

ISBN
1861976127    4
0688121195    4
0890438439    4
0006548830    1
0812510240    1
0681626836    1
0696212536    1
0698111435    1
0739404512    1
0743223926    1
0743410467    1
0785328629    1
0789423936    1
0805054537    1
0810322951    1
0812542967    1
0671793888    1
0812577663    1
0823016560    1
0826317979    1
0836213122    1
0895779234    1
1558582312    1
156231419X    1
1565122259    1
1578630053    1
1583220577    1
0679729976    1
0671735837    1
0060927569    1
0671220519    1
0061094048    1
0099284677    1
0140081038    1
0140234012    1
0224051261    1
0312198876    1
0312255012    1
0312857055    1
0330375466    1
0340832878    1
0345323750    1
0380720795    1
0380750074    1
0399128905    1
0399150412    1
042516358X    1
0500201862    1
0500511160    1
0553280120    1
0553282220    1
0553346326    1
0590438786    1
0670030392    1
0670858692    1
0671009109    1
1882606450    1
Name: Book-Rating, dtype: int64

In [ ]:
# 책 제목도 df에 추가하기
count_knn_rec = pd.merge(friend_count_df, book_info[['ISBN', 'Book-Title']], on = 'ISBN')
count_knn_rec

,ISBN,Book-Rating,Book-Title
0,1861976127,4,"Eats, Shoots and Leaves: The Zero Tolerance Ap..."
1,0688121195,4,Voluntary Simplicity (rev) : Toward A Way Of L...
2,0890438439,4,How to Clean Practically Anything
3,0006548830,1,Information
4,0812510240,1,Firedance
5,0681626836,1,Americana: Roadside Memories
6,0696212536,1,The Home Depot Landscaping 1-2-3: Regional Edi...
7,0698111435,1,Prizzi's Honor
8,0739404512,1,Houseplant Encyclopedia
9,0743223926,1,The Seven Year Hitch : A Family Odyssey


In [ ]:
#평점 기반 추천 - 평점이 가장 좋은 책
friend_avg_df = friend_df.groupby('ISBN').agg('mean').sort_values(
    'Book-Rating', ascending = False)['Book-Rating'] #깔끔하게 df로 표현하기 위해 Book-Rating만 가져오기
friend_avg_df

ISBN
0739404512    10.00
0836213122    10.00
0553346326    10.00
0671220519    10.00
0679729976    10.00
0380750074    10.00
0696212536    10.00
0671009109    10.00
0670030392     9.00
0895779234     9.00
1558582312     9.00
042516358X     9.00
0671793888     9.00
0500511160     9.00
0060927569     9.00
0099284677     9.00
1882606450     9.00
1861976127     8.50
0826317979     8.00
0823016560     8.00
0805054537     8.00
0743223926     8.00
0681626836     8.00
1565122259     8.00
0006548830     8.00
0140081038     8.00
0500201862     8.00
0399128905     8.00
0688121195     7.75
0670858692     7.00
0789423936     7.00
1583220577     7.00
156231419X     7.00
0140234012     7.00
0224051261     7.00
0312198876     7.00
0590438786     7.00
0810322951     7.00
0553282220     7.00
0698111435     7.00
0553280120     7.00
0890438439     6.25
0345323750     6.00
0399150412     6.00
0330375466     6.00
0340832878     6.00
0812510240     6.00
0812542967     6.00
0743410467     6.00
0061094048     

In [ ]:
# 책 제목도 df에 추가하기
avg_knn_rec = pd.merge(friend_avg_df, book_info[['ISBN', 'Book-Title']], on = 'ISBN')
avg_knn_rec

,ISBN,Book-Rating,Book-Title
0,0739404512,10.00,Houseplant Encyclopedia
1,0836213122,10.00,There's Treasure Everywhere--A Calvin and Hobb...
2,0553346326,10.00,Infant Massage: A Handbook for Loving Parents
3,0671220519,10.00,Consumer guide rating &amp; raising indoor pla...
4,0679729976,10.00,The Stories of Vladimir Nabokov
5,0380750074,10.00,The Makeover Club
6,0696212536,10.00,The Home Depot Landscaping 1-2-3: Regional Edi...
7,0671009109,10.00,ANOTHER LIFE
8,0670030392,9.00,New Herb Bible: Growing and Knowing Your Herbs...
9,0895779234,9.00,"A-Z of Annuals, Biennials &amp; Bulbs (Success..."


In [ ]:
#count df와 avg df를 concat을 함으로서 최종적인 결과값을 보기 쉽게 표현
result_df = pd.concat([friend_count_df, friend_avg_df], axis=1)
result_df.columns = ['Count', 'Average']
result_df = pd.merge(result_df, book_info[['ISBN','Book-Title']], on = 'ISBN') #제목 추가
result_df.head(10) #탑 10

,ISBN,Count,Average,Book-Title
0,1861976127,4,8.50,"Eats, Shoots and Leaves: The Zero Tolerance Ap..."
1,0688121195,4,7.75,Voluntary Simplicity (rev) : Toward A Way Of L...
2,0890438439,4,6.25,How to Clean Practically Anything
3,0006548830,1,8.00,Information
4,0812510240,1,6.00,Firedance
5,0681626836,1,8.00,Americana: Roadside Memories
6,0696212536,1,10.00,The Home Depot Landscaping 1-2-3: Regional Edi...
7,0698111435,1,7.00,Prizzi's Honor
8,0739404512,1,10.00,Houseplant Encyclopedia
9,0743223926,1,8.00,The Seven Year Hitch : A Family Odyssey


In [ ]:
#타겟 유저가 누구였는지 다시 확인
target_user_id

88705

### 결과값 도출하기

In [ ]:
#여기서 이미 본 것을 지워준 뒤 결과값 도출
unseen_book = set(result_df['ISBN'].to_list()) - set(target_user_df['ISBN'].to_list())
result_rec_df = result_df[result_df['ISBN'].isin(unseen_book)].sort_values(['Count','Average'], ascending = False)
result_rec_df.head()

,ISBN,Count,Average,Book-Title
0,1861976127,4,8.50,"Eats, Shoots and Leaves: The Zero Tolerance Ap..."
1,0688121195,4,7.75,Voluntary Simplicity (rev) : Toward A Way Of L...
2,0890438439,4,6.25,How to Clean Practically Anything
6,0696212536,1,10.00,The Home Depot Landscaping 1-2-3: Regional Edi...
8,0739404512,1,10.00,Houseplant Encyclopedia


In [ ]:
result_rec_df.drop(columns = ['Count','Average'])[:10]

,ISBN,Book-Title
0,1861976127,"Eats, Shoots and Leaves: The Zero Tolerance Ap..."
1,0688121195,Voluntary Simplicity (rev) : Toward A Way Of L...
2,0890438439,How to Clean Practically Anything
6,0696212536,The Home Depot Landscaping 1-2-3: Regional Edi...
8,0739404512,Houseplant Encyclopedia
20,0836213122,There's Treasure Everywhere--A Calvin and Hobb...
27,0679729976,The Stories of Vladimir Nabokov
30,0671220519,Consumer guide rating &amp; raising indoor pla...
43,0380750074,The Makeover Club
51,0553346326,Infant Massage: A Handbook for Loving Parents
